### Importing Packages



In [14]:

import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from xgboost import XGBClassifier
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from scipy.stats import mode
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

### Load Data

In [22]:

data=pd.read_csv('/Users/kaku/Downloads/AIML/Crashathon/Accident_train.csv')
data.describe()

,Collision_Ref_No,Collision_Severity,Day_of_Collision,Month_of_Collision,Hour_of_Collision,Carriageway_Type,Speed_Limit,Junction_Detail,Junction_Control,Ped_Crossing_HC,Ped_Crossing_PC,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site
count,10043.000000,10043.000000,10043.000000,10043.000000,9759.000000,10043.000000,10043.000000,9750.000000,9744.000000,9722.000000,9743.000000,10043.000000,10043.000000,9734.000000,9755.000000
mean,6565.855720,2.880613,15.662153,6.655382,13.624859,12.201036,42.415613,6.400410,4.158764,1.007714,2.231243,3.237778,4.717216,1.661804,1.057099
std,3928.749772,0.359226,8.783339,3.526781,5.099759,2.744209,14.524589,5.016853,2.864940,0.114938,3.252212,2.067334,3.745739,1.470371,0.480414
min,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,3060.500000,3.000000,8.000000,4.000000,10.000000,13.000000,30.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,7009.000000,3.000000,16.000000,7.000000,14.000000,13.000000,30.000000,6.000000,4.000000,1.000000,1.000000,3.000000,3.000000,1.000000,1.000000
75%,10078.500000,3.000000,23.000000,10.000000,17.000000,13.000000,60.000000,12.000000,7.000000,1.000000,1.000000,5.000000,9.000000,2.000000,1.000000
max,13147.000000,3.000000,31.000000,12.000000,23.000000,14.000000,70.000000,12.000000,7.000000,3.000000,12.000000,7.000000,10.000000,10.000000,6.000000


In [23]:
#data['Special_Conditions_at_Site'].value_counts()

### Data Preprocessing

In [24]:
fin_data=data.drop(['Collision_Ref_No'], axis=1)
fin_data.shape

(10043, 16)

In [32]:
#Shifting Labels to the last
cs = fin_data['Collision_Severity']
fin_data.drop(labels=['Collision_Severity'], axis=1,inplace = True)
fin_data.insert(12, 'Collision_Severity', cs)


In [ ]:
##ONe hot
hot_weekday=pd.get_dummies(fin_data['Weekday_of_Collision'])
hot_areas=pd.get_dummies(fin_data['Policing_Area'])
#fin_data['Weekday_of_Collision'].unique()

In [ ]:
fin_data=pd.concat([hot_areas,hot_weekday,fin_data], axis=1)
#print(list(large_data),large_data.shape)

In [25]:
###discretisation
fin_data['Policing'] = fin_data.Policing_Area.astype('category').cat.rename_categories(range(1, fin_data.Policing_Area.nunique()+1))

week={'MON':1,'TUE':2,'WED':3,'THU':4,'FRI':5,'SAT':6,'SUN':7}
fin_data["Weekday_of_Collision"] = fin_data["Weekday_of_Collision"].map(week)

#print(list(fin_data),fin_data.shape)
fin_data.shape

(10043, 17)

In [27]:
fin_data=fin_data.drop(['Policing_Area','Ped_Crossing_HC','Road_Surface_Conditions','Special_Conditions_at_Site'], axis=1)

In [33]:
fin_data.head()

,Weekday_of_Collision,Day_of_Collision,Month_of_Collision,Hour_of_Collision,Carriageway_Type,Speed_Limit,Junction_Detail,Junction_Control,Ped_Crossing_PC,Light_Conditions,Weather_Conditions,Policing,Collision_Severity
0,1,4,8,14.0,13,60,1.0,1.0,1.0,2,9,20,3
1,6,8,8,17.0,11,50,12.0,7.0,1.0,4,3,8,3
2,3,5,11,17.0,1,60,12.0,7.0,1.0,2,2,29,3
3,7,18,10,16.0,12,70,6.0,NaN,1.0,1,3,8,3
4,1,23,11,9.0,13,60,6.0,7.0,1.0,2,3,33,3


In [34]:
##columns containing Nan values
fin_data.columns[fin_data.isna().any()].tolist()
#large_data.loc[:, large_data.isna().any()]

['Hour_of_Collision',
 'Junction_Detail',
 'Junction_Control',
 'Ped_Crossing_PC',
 'Policing']

In [35]:
#large_data.max()

In [36]:
#x=mode(large_data)


In [37]:
#(x[0])

In [38]:
##filling Nan with modes
fin_data_full=fin_data.apply(lambda x:x.fillna(x.value_counts().index[0]))

In [39]:
#large_data_full.loc[:, large_data.isna().any()]
fin_data_full.columns[fin_data_full.isna().any()].tolist()
#large_data.isna().any()
#large_data_full

[]

In [ ]:
#large_data_full.describe()
fin_data_full=fin_data_full.drop(['Ped_Crossing_HC','Special_Conditions_at_Site'],axis=1)

In [83]:
fin_data_full.describe()

,Weekday_of_Collision,Day_of_Collision,Month_of_Collision,Hour_of_Collision,Carriageway_Type,Speed_Limit,Junction_Detail,Junction_Control,Ped_Crossing_PC,Light_Conditions,Weather_Conditions,Collision_Severity
count,10043.000000,10043.000000,10043.000000,10043.000000,10043.000000,10043.000000,10043.000000,10043.000000,10043.000000,10043.000000,10043.000000,10043.000000
mean,3.875933,15.662153,6.655382,13.720303,12.201036,42.415613,6.242856,4.243354,2.194464,3.237778,4.717216,2.880613
std,1.907214,8.783339,3.526781,5.058169,2.744209,14.524589,5.025990,2.862988,3.210115,2.067334,3.745739,0.359226
min,1.000000,1.000000,1.000000,0.000000,1.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2.000000,8.000000,4.000000,10.000000,13.000000,30.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000
50%,4.000000,16.000000,7.000000,14.000000,13.000000,30.000000,6.000000,7.000000,1.000000,3.000000,3.000000,3.000000
75%,5.000000,23.000000,10.000000,17.000000,13.000000,60.000000,12.000000,7.000000,1.000000,5.000000,9.000000,3.000000
max,7.000000,31.000000,12.000000,23.000000,14.000000,70.000000,12.000000,7.000000,12.000000,7.000000,10.000000,3.000000


### Data Splitting 

###Separating the data and labels

In [41]:
X=fin_data_full.values[:,:12]
X.shape

(10043, 12)

In [ ]:
#large_data_full['Collision_Severity'].value_counts()

In [42]:
#Y=list(fin_data_full.values[:,15:].flatten())
Y=fin_data_full.values[:,12:].flatten()
type(Y)

numpy.ndarray

###Splitting using Cross-Validation techniques

In [ ]:
kf=KFold(n_splits=50, shuffle=False, random_state=100)
kf.get_n_splits(fin_data_full)

In [ ]:
for train_index, test_index in kf.split(X):
    train, val = X[train_index], X[test_index]
    ltrain, lval = Y[train_index], Y[test_index]
    #print(val.shape,lval.shape)
    clf.fit(train, list(ltrain))
    Y_pred = clf.predict(val)
    accuracy = accuracy_score(list(lval),Y_pred)
    print(accuracy)
    


In [ ]:
#scores = cross_val_score(bbc, Xre, Yre, cv=50)
#scores

In [ ]:

#cv = ShuffleSplit(n_splits=50, test_size=0.29, random_state=102)
#cross_val_score(bbc, Xre, Yre, cv=cv)

###Splitting into train and val using sklearn splitter

In [ ]:
#X_train, X_val, Y_train, Y_val = train_test_split(X,Y,test_size=0.20,random_state=400)
#print(X_train.shape,X_val.shape,len(Y_train),len(Y_val))

### Training Classifiers

In [43]:


#clf=DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=20, min_samples_leaf=500)
#clf = RandomForestClassifier(class_weight={1:3,2:3,3:1},criterion='gini',min_samples_leaf=5,max_depth=10, random_state=50,n_estimators=70)
#clf=AdaBoostClassifier(learning_rate=0.1,random_state=100,n_estimators=100)
#clf=XGBClassifier(learning_rate=0.01,random_state=100)
#clf=GradientBoostingClassifier(learning_rate=0.01,max_depth=30,n_estimators=100)
#clf.fit(X, Y)

In [68]:
parameters={'criterion':['gini'],'min_samples_leaf':range(1,50,5),'max_depth':range(1,25,5),'random_state':range(0,400,50),'n_estimators':range(40,70,5)}

In [69]:
grid_search=GridSearchCV(clf,parameters,cv=None)

In [70]:
grid_search.fit(X,list(Y))

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'criterion': ['gini'], 'min_samples_leaf': range(1, 50, 5), 'max_depth': range(1, 25, 5), 'random_state': range(0, 400, 50), 'n_estimators': range(40, 70, 5)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [72]:
grid_search.best_score_,grid_search.best_params_

(0.8952504231803246,
 {'criterion': 'gini',
  'max_depth': 11,
  'min_samples_leaf': 1,
  'n_estimators': 45,
  'random_state': 350})

In [132]:
clf = RandomForestClassifier(criterion='gini',min_samples_leaf=1,max_depth=11, random_state=350,n_estimators=45)
clf.fit(X, list(Y))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=11, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=45, n_jobs=1,
            oob_score=False, random_state=350, verbose=0, warm_start=False)

In [ ]:
Y_pred = bbc.predict(X_val)
accuracy = accuracy_score(Y_val,Y_pred)
print(accuracy)

### Sampling to remove Bias

In [ ]:
#from imblearn.over_sampling import SMOTE
#sm=SMOTE(ratio='auto', kind='regular')
#Xre, Yre= sm.fit_sample(X, Y)


In [ ]:
#from imblearn.over_sampling import RandomOverSampler
#ros = RandomOverSampler(random_state=100)
#Xre, Yre= ros.fit_sample(X, Y)

In [ ]:
#from imblearn.under_sampling import NearMiss
#nm = NearMiss(random_state=50)
#Xre, Yre= nm.fit_sample(X, Y)

In [ ]:
#from imblearn.ensemble import BalancedBaggingClassifier
#bbc = BalancedBaggingClassifier(random_state=42,bootstrap_features=False,n_estimators=65,max_features=14)
#bbc.fit(X_train, Y_train)

In [ ]:
#unique, counts = np.unique(Yre, return_counts=True)
#print (np.asarray((unique, counts)).T)

### Testing

In [115]:
test=pd.read_csv('/Users/kaku/Downloads/AIML/Crashathon/Accident_test.csv')

In [116]:
#test

In [117]:
test_data=test.drop(['Collision_Ref_No','Collision_Severity'],axis=1)

In [118]:
test_data.shape

(354, 15)

In [119]:
#hot_weekday_test=pd.get_dummies(test_data['Weekday_of_Collision'])
#hot_areas_test=pd.get_dummies(test_data['Policing_Area'])

In [120]:
#test_data=pd.concat([hot_areas_test,hot_weekday_test,test_data], axis=1)
#print(list(test_data),test_data.shape)

In [121]:
test_data['Policing'] = test_data.Policing_Area.astype('category').cat.rename_categories(range(1, test_data.Policing_Area.nunique()+1))

week={'MON':1,'TUE':2,'WED':3,'THU':4,'FRI':5,'SAT':6,'SUN':7}
test_data["Weekday_of_Collision"] = test_data["Weekday_of_Collision"].map(week)

#print(list(fin_data),fin_data.shape)
test_data.head()



,Policing_Area,Weekday_of_Collision,Day_of_Collision,Month_of_Collision,Hour_of_Collision,Carriageway_Type,Speed_Limit,Junction_Detail,Junction_Control,Ped_Crossing_HC,Ped_Crossing_PC,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Policing
0,DAST,1,17,2,21.0,13,60,10.0,7.0,1.0,1.0,5,9,6.0,1.0,21
1,ARBC,2,13,1,8.0,13,60,10.0,NaN,1.0,1.0,7,1,1.0,1.0,3
2,NEMD,6,24,10,18.0,13,30,6.0,7.0,1.0,1.0,6,1,1.0,1.0,37
3,MEAN,7,10,5,13.0,11,70,12.0,7.0,1.0,1.0,1,9,1.0,1.0,31
4,FOYL,4,23,1,10.0,13,60,1.0,1.0,1.0,1.0,2,3,2.0,1.0,26


In [114]:
fin_data_full.head()

,Weekday_of_Collision,Day_of_Collision,Month_of_Collision,Hour_of_Collision,Carriageway_Type,Speed_Limit,Junction_Detail,Junction_Control,Ped_Crossing_PC,Light_Conditions,Weather_Conditions,Policing,Collision_Severity
0,1,4,8,14.0,13,60,1.0,1.0,1.0,2,9,20,3
1,6,8,8,17.0,11,50,12.0,7.0,1.0,4,3,8,3
2,3,5,11,17.0,1,60,12.0,7.0,1.0,2,2,29,3
3,7,18,10,16.0,12,70,6.0,7.0,1.0,1,3,8,3
4,1,23,11,9.0,13,60,6.0,7.0,1.0,2,3,33,3


In [122]:
test_data=test_data.drop(['Road_Surface_Conditions','Policing_Area','Ped_Crossing_HC','Special_Conditions_at_Site'], axis=1)
#print(large_data)
#print(list(test_data),test_data.shape)

In [124]:
test_data.head()

,Weekday_of_Collision,Day_of_Collision,Month_of_Collision,Hour_of_Collision,Carriageway_Type,Speed_Limit,Junction_Detail,Junction_Control,Ped_Crossing_PC,Light_Conditions,Weather_Conditions,Policing
0,1,17,2,21.0,13,60,10.0,7.0,1.0,5,9,21
1,2,13,1,8.0,13,60,10.0,NaN,1.0,7,1,3
2,6,24,10,18.0,13,30,6.0,7.0,1.0,6,1,37
3,7,10,5,13.0,11,70,12.0,7.0,1.0,1,9,31
4,4,23,1,10.0,13,60,1.0,1.0,1.0,2,3,26


In [125]:
test_data.columns[test_data.isna().any()].tolist()

['Hour_of_Collision',
 'Junction_Detail',
 'Junction_Control',
 'Ped_Crossing_PC',
 'Policing']

In [126]:
test_data_full=test_data.apply(lambda x:x.fillna(x.value_counts().index[0]))

In [127]:
test_data_full.columns[test_data_full.isna().any()].tolist()

[]

In [128]:
test_data_full.shape

(354, 12)

In [ ]:
Y_pred_test = clf.predict(test_data_full)
Y_pred_test

### File Saving 

In [ ]:
'''q=pd.DataFrame(Y_pred_test)
q.index += 1 
q.to_csv('Take_1.csv',sep=',',index=True,index_label='S.No.', header=['Collision_Severity'])'''



In [134]:
fil=open('Take_GSCV.csv','w')
fil.write('S.No.,Collision_Severity\n')
for i in range(Y_pred_test.shape[0]-1):
   fil.write('%d,%d\n'%(i+1,Y_pred_test[i]))
fil.write('%d,%d'%(i+2,Y_pred_test[i+1]))
fil.close()

In [ ]:
#from sklearn import tree
#tree.export_graphviz(clf, out_file='clf.dot')